# Replay demonstration

In [ ]:
%reset -f
%matplotlib inline
from replay import plot, smln

p = {
    # Exc. PC membrane
    'T_M_PC': 0.05, 'E_L_PC': -0.068, 'V_TH_PC': -0.036, 'V_R_PC': -0.068, 'T_R_PC': 0.008,
    'E_AHP_PC': -0.07, 'T_AHP_PC': 0.05, 'W_AHP_PC': 0.0,
    
    # Exc. PC architecture
    'N_PC': 1000,
    'Z_PC_PC': 1.19, 'L_PC_PC': 0.084,
    'W_E_PC_PC': 0.024, 'S_E_PC_PC': 0,
    
    # "Place-tuned" PC inputs
    'R_MAX': 600, 'L_PL': 0.2, 'B_S': 0.01, 'S_TH': 0.1, 
    'W_E_PC_PL': 0.02, 'S_E_PC_PL': 0,
    
    # Inh. membrane
    'T_M_INH': 0.009, 'E_L_INH': -0.058, 'V_TH_INH': -0.036, 'V_R_INH': -0.058, 'T_R_INH': 0.002,  # unused (presently)
    
    # Inh. architecture
    'N_INH': 100,
    'Z_S_PC_INH': 1,  'L_S_PC_INH': 0.1,  # unused (presently)
    'Z_C_PC_INH': 1, 'L_C_PC_INH': 0.1,  # unused (presently)
    'W_E_INH_PC': 0., 'S_E_INH_PC': 0,   # unused (presently)
    
    'Z_INH_PC': 0., 'L_INH_PC': 0.1,  # unused (presently)
    'W_I_PC_INH': 0., 'S_I_PC_INH': 0.,  # unused (presently)
    
    # initial distr of gating input weights
    'W_E_INIT_PC_ST': 0.008216, 'S_E_INIT_PC_ST': 0
    
    # synaptic reversal potentials & time constants
    'E_E': 0, 'E_I': -0.08, 'T_E': 0.002, 'T_I': 0.005, 
    
    # Gating input firing rates for traj and replay epochs
    'FR_TRJ_PC_ST': 3, 'FR_RPL_PC_ST': 127, 
    
    # trigger 
    'A_TR': 0.008, 'R_TR': 0.4, 'D_T_TR': 0.003, 
    
    # plasticity
    'A_P': 2,
    'T_C': 1, 'T_W': 1,   # unused (remove)
    'C_S': 1, 'B_C': 1,   # unused (remove)
}

s_params = {
    "DT": 0.0005, "BOX_H": 2, "BOX_W": 2, "END_X": 1, "SPEED": 0.2, "X_TRG": 1, "Y_TRG": -0.75,
    "TURN_X": 0, "TURN_Y": -0.75, "START_X": -1, "START_Y": 0.75,
    "metrics": {
        "WDW": 0.1, "PITCH": 10, "RADIUS": 0.2, "MIN_SCALE_TRJ": 1.25,
        "MIN_FRAC_SPK_TRJ": 0.75, "MAX_AVG_SPK_CT_TRJ": 3, "TRJ_NON_TRJ_SPK_RATIO": 1.5
    },
    "RNG_SEED": 2,
    "schedule": {"SMLN_DUR": 22, "TRG_START_T": 21, "TRJ_START_T": 1, "REPLAY_EPOCH_START_T": 20}}

apxn = True

print('Running smln...')
rslt = smln.run(p, s_params, apxn)
print('Smln complete.')

# make plots
plot.heat_maps(rslt, (1, 1.15));

plot.raster(
    rslt,
    xys=[
    (-.75, .75), (0, .75), (0, 0), (0, -.75), (.75, -.75),  # trj
    (-.75, 0), (-.75, -.75), (.75, .75), (.75, 0),  # non-trj
    ],
    nearest=3,
    epoch='replay');

plot.raster(
    rslt,
    xys=[
    (-.75, .75), (0, .75), (0, 0), (0, -.75), (.75, -.75),  # trj
    (-.75, 0), (-.75, -.75), (.75, .75), (.75, 0),  # non-trj
    ],
    nearest=3,
    epoch=(1, 1.15));